In [1]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [5]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [6]:
# 오늘 날짜
today = datetime.today().strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제가 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')

#### 종합쇼핑몰 납품상세내역

In [7]:
all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_all', bigquerykey_path)

In [8]:
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '수요기관코드', '수요기관구분', '수요기관지역명','납품요구지청명']]
all_shop_df_fin = all_shop_df_fin.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [9]:
# 특수문자, 숫자, 영어 제거 함수 (연속된 공백을 단일 공백으로 변환 포함)
def clean_text(text):
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', text)
    text = re.sub('\s+', ' ', text).strip()  # 연속된 공백을 단일 공백으로 변환
    return text

In [10]:
# 사업명 전처리
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명'].apply(clean_text)
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.strip()

In [11]:
# 수요기관 필터링
dist_nm = ('|').join(all_shop_df_fin[all_shop_df_fin['수요기관지역명'].str.split(' ').str[1].notnull()]['수요기관지역명'].str.split(' ').str[1])

In [12]:
# 사업명 split 키워드 필터링
mapping_keywd_all = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' '))) ## 전체 키워드
mapping_keywd_all_filter_cnt = pd.Series(Counter(mapping_keywd_all))

mapping_keywd3 = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' ').str[:3])) ## split 3개 키워드
mapping_keywd3_filter_cnt = pd.Series(Counter(mapping_keywd3))

mapping_keywd2 = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' ').str[:2])) ## split 2개 키워드
mapping_keywd2_filter_cnt = pd.Series(Counter(mapping_keywd2))

mapping_keywd1 = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' ').str[:1])) ## split 1개 키워드
mapping_keywd1_filter_cnt = pd.Series(Counter(mapping_keywd1))

mapping_keywd_all_filter_cnt_nm = ('|').join(mapping_keywd_all_filter_cnt[mapping_keywd_all_filter_cnt <= mapping_keywd_all_filter_cnt.mean()].keys())
mapping_keywd3_filter_cnt_nm = ('|').join(mapping_keywd3_filter_cnt[mapping_keywd3_filter_cnt <= mapping_keywd3_filter_cnt.mean()].keys())
mapping_keywd2_filter_cnt_nm = ('|').join(mapping_keywd2_filter_cnt[mapping_keywd2_filter_cnt <= mapping_keywd2_filter_cnt.mean()].keys())
mapping_keywd1_filter_cnt_nm = ('|').join(mapping_keywd1_filter_cnt[mapping_keywd1_filter_cnt <= mapping_keywd1_filter_cnt.mean()].keys())

In [13]:
# 전체 키워드 데이터셋
mapping_keywd_df = pd.DataFrame(Counter(mapping_keywd_all).items(),columns=['키워드','빈도수'])
mapping_keywd_df = mapping_keywd_df[mapping_keywd_df['키워드'].apply(len)!=1].reset_index(drop=True)
mapping_keywd_df.sort_values('빈도수',ascending=False)

,키워드,빈도수
4,인조잔디,343
16,관급자재,219
9,구입,175
8,관급자재인조잔디,168
34,구매,103
...,...,...
577,노후시설물,1
576,사염공원,1
575,부안초,1
573,강원외국어,1


In [14]:
# 종합쇼핑몰 기준 중요키워드 추출
print('1등급 키워드 : ', list(mapping_keywd_df[mapping_keywd_df['빈도수'] >= 50].sort_values('빈도수',ascending=False)['키워드']))
print('2등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 50) & (mapping_keywd_df['빈도수'] >= 40)].sort_values('빈도수',ascending=False)['키워드']))
print('3등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 40) & (mapping_keywd_df['빈도수'] >= 30)].sort_values('빈도수',ascending=False)['키워드']))
print('4등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 30) & (mapping_keywd_df['빈도수'] >= 20)].sort_values('빈도수',ascending=False)['키워드']))
print('5등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 20) & (mapping_keywd_df['빈도수'] >= 10)].sort_values('빈도수',ascending=False)['키워드']))
print('기타 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 10)].sort_values('빈도수',ascending=False)['키워드']))

1등급 키워드 :  ['인조잔디', '관급자재', '구입', '관급자재인조잔디', '구매', '운동장', '테니스장', '조성공사', '게이트볼장', '공사', '체육시설']
2등급 키워드 :  []
3등급 키워드 :  ['정비공사', '조성사업', '교체공사', '설치', '관급인조잔디', '축구장']
4등급 키워드 :  ['풋살장', '개선공사', '조성', '파크골프장', '구입인조잔디', '족구장']
5등급 키워드 :  ['환경개선공사', '다목적구장', '교체', '정비사업', '보수공사', '조성공사인조잔디', '관급', '교체공사인조잔디', '설치공사', '환경개선', '연병장', '다목적', '부대', '정비공사인조잔디', '체육공원']
기타 키워드 :  ['시설개선사업', '신축', '생활체육공원', '생활체육시설', '개보수공사', '조달구매', '조달', '개보수', '시설개선공사', '시설', '교체사업', '따른', '개소', '개선', '추가', '어린이공원', '노후시설', '정비', '시설개선', '위한', '공공하수처리시설', '관련', '요청', '본부', '실내게이트볼장', '트랙조성', '사업', '관급자재조경인조잔디', '증설공사', '설치공사인조잔디', '인조잔디운동장', '리모델링', '당진', '체육', '개선사업', '김영훈', '하부', '공사인조잔디', '관급자재인조잔디구입', '학교운동장', '비가림시설', '시행', '일원', '여단', '물품', '구매인조잔디', '보수', '배드민턴장', '조성사업인조잔디', '그라운드골프장', '시설물', '근린공원', '구매설치', '본원', '옥외', '환경개선사업', '수해복구공사', '수지아르피아', '잔디', '환경개선인조잔디', '교체관급인조잔디', '확장', '재조성', '조달구입', '운동공간', '환경친화적', '트랙보수', '사단', '스포츠파크', '야구장', '노후인조잔디', '토목', '그린스마트', '기타시설공사', '관급자', '인조잔디구장',

In [15]:
# 빈도수 키워드
freq_kwd = ('|').join(list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 10)]['키워드']))

#### 지자체 세부사업별 예산서

In [16]:
# 빅쿼리에서 불러오기
budget_df_today = get_dataframe_from_bigquery('budget','budget_df_0' + today,bigquerykey_path)
budget_df_ytday = get_dataframe_from_bigquery('budget','budget_df_0' + ytday,bigquerykey_path)

In [17]:
# 필요키워드1
need_kwd1 = '인조잔디|초등학교|중학교|고등학교|대학교|운동장|다목적|종합운동장|체육공원|체육시설|체육센터|스포츠센터|관급|연병장|족구장|축구장|야구장|풋살장|배드민턴장|테니스장|게이트볼장|하키장' 

# 필요키워드2
need_kwd2 = '개선|교체|공사|구매|구입|설치|정비|조성'

# 제외키워드
except_kwd = '경관|조명|펜스|복지|교육|학습|보건|행사|예방|안전|치안|창출|활성화|유아|검진|조명|오염|예술|급식|보상|주차장|개방|CCTV|미술|교실|관리|운영|자전거|울타리|버스|맨홀|노점|노상|하천|하수도|광고|약수터|전통시장|계단|정화조|도로확장|골프장|입학|도로개설|소프트웨어|가속기|진로진학|인터넷|음악|취업|역량|억제제|살포|급수관|하수관로|요양기관|도로방음벽|가로등|도로개설|보행자도로|진입도로|철거'

In [18]:
# 키워드 필터링
budget_df_today = budget_df_today[budget_df_today['세부사업명'].str.contains(need_kwd1) & ~budget_df_today['세부사업명'].str.contains(except_kwd)].reset_index(drop=True)
budget_df_ytday = budget_df_ytday[budget_df_ytday['세부사업명'].str.contains(need_kwd1) & ~budget_df_ytday['세부사업명'].str.contains(except_kwd)].reset_index(drop=True)

In [19]:
# 종료 사업
budget_df_delete_temp1 = budget_df_ytday[budget_df_ytday['세부사업코드'].isin(set(budget_df_ytday['세부사업코드']) - set(budget_df_today['세부사업코드']))].reset_index(drop=True)
budget_df_delete_temp2 = budget_df_ytday[budget_df_ytday['세부사업명'].isin(set(budget_df_ytday['세부사업명']) - set(budget_df_today['세부사업명']))].reset_index(drop=True)
budget_df_delete = pd.concat([budget_df_delete_temp1,budget_df_delete_temp2],axis=0).drop_duplicates().reset_index(drop=True)
budget_df_delete

,회계연도,지역코드,지역명,자치단체코드,자치단체명,회계구분명,세부사업코드,세부사업명,집행일자,예산현액,지출액,편성액,분야명,부문명,행정자치단체코드
0,2024,1100000,서울,1114000,서울성동구,일반회계,30300002023301E1,실외 어린이체육시설 조성,20240702,570000000,239700000,0,문화및관광,체육,3030000


In [20]:
# 새로 추가된 사업
budget_df_new_temp1 = budget_df_today[~budget_df_today['세부사업코드'].isin(budget_df_ytday['세부사업코드'])].reset_index(drop=True).reset_index(drop=True)
budget_df_new_temp2 = budget_df_today[~budget_df_today['세부사업명'].isin(budget_df_ytday['세부사업명'])].reset_index(drop=True).reset_index(drop=True)
budget_df_new = pd.concat([budget_df_new_temp1,budget_df_new_temp2],axis=0).drop_duplicates().reset_index(drop=True)
budget_df_new

,회계연도,지역코드,지역명,자치단체코드,자치단체명,회계구분명,세부사업코드,세부사업명,집행일자,예산현액,지출액,편성액,분야명,부문명,행정자치단체코드
0,2024,1100000,서울,1124000,서울마포구,일반회계,3130000202430056,디지털 실내체육시설 설치,20240703,14000000,0,14000000,사회복지,보육ㆍ가족및여성,3130000
1,2024,4100000,경기,4175000,경기연천군,체육진흥기금,4140000202430171,노후 체육시설 정비,20240703,260000000,0,260000000,문화및관광,체육,4140000
2,2024,1100000,서울,1114000,서울성동구,일반회계,3030000202130081,공공체육시설 개보수 지원(보조),20240703,349300000,300000,349300000,문화및관광,체육,3030000
3,2024,1100000,서울,1114000,서울성동구,일반회계,30300002023301E1,겨울철 실외 체육시설 조성,20240703,1070000000,239700000,500000000,문화및관광,체육,3030000


In [21]:
# 사업명 전처리
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명'].apply(clean_text)
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.strip()

# budget_df_ytday['세부사업명_re'] = budget_df_ytday['세부사업명'].apply(clean_text)
# budget_df_ytday['세부사업명_re'] = budget_df_ytday['세부사업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
# budget_df_ytday['세부사업명_re'] = budget_df_ytday['세부사업명_re'].str.strip()

In [23]:
# 해당 지역 추출
filtered_budget_df_temp1 = budget_df_today[budget_df_today['자치단체명'].str.contains(dist_nm)].reset_index(drop=True)
filtered_budget_df_temp2 = budget_df_today[budget_df_today['자치단체명'].str.contains('본청')].reset_index(drop=True)

filtered_budget_df_today = pd.concat([filtered_budget_df_temp1,filtered_budget_df_temp2],axis=0).reset_index(drop=True)
# filtered_budget_df_ytday = budget_df_ytday[budget_df_ytday['자치단체명'].str.contains(dist_nm)].reset_index(drop=True)

In [24]:
# split 키워드 필터링
filtered_budget_df_today = filtered_budget_df_today[filtered_budget_df_today['세부사업명_re'].str.contains(mapping_keywd_all_filter_cnt_nm)] ## 전체 키워드
filtered_budget_df_today = filtered_budget_df_today[filtered_budget_df_today['세부사업명_re'].str.contains(mapping_keywd3_filter_cnt_nm)] ## split 3개 키워드
filtered_budget_df_today = filtered_budget_df_today[filtered_budget_df_today['세부사업명_re'].str.contains(mapping_keywd2_filter_cnt_nm)] ## split 2개 키워드
filtered_budget_df_today = filtered_budget_df_today[filtered_budget_df_today['세부사업명_re'].str.contains(mapping_keywd1_filter_cnt_nm)] ## split 1개 키워드

# filtered_budget_df_ytday = filtered_budget_df_ytday[filtered_budget_df_ytday['세부사업명_re'].str.contains(mapping_keywd_all_filter_cnt_nm)] ## 전체 키워드
# filtered_budget_df_ytday = filtered_budget_df_ytday[filtered_budget_df_ytday['세부사업명_re'].str.contains(mapping_keywd3_filter_cnt_nm)] ## split 3개 키워드
# filtered_budget_df_ytday = filtered_budget_df_ytday[filtered_budget_df_ytday['세부사업명_re'].str.contains(mapping_keywd2_filter_cnt_nm)] ## split 2개 키워드
# filtered_budget_df_ytday = filtered_budget_df_ytday[filtered_budget_df_ytday['세부사업명_re'].str.contains(mapping_keywd1_filter_cnt_nm)] ## split 1개 키워드

In [25]:
# 종합쇼핑몰 기준 필터링
bid_cd_split_today = list(budget_df_today[budget_df_today['세부사업코드'].isin(filtered_budget_df_today['세부사업코드'])]['세부사업코드']) # split 키워드 필터링
bid_cd_freq_today = list(budget_df_today[budget_df_today['세부사업명'].str.contains(freq_kwd)]['세부사업코드']) # 빈도수 키워드 필터링

# bid_cd_split_ytday = list(budget_df_ytday[budget_df_ytday['세부사업코드'].isin(filtered_budget_df_ytday['세부사업코드'])]['세부사업코드']) # split 키워드 필터링
# bid_cd_freq_ytday = list(budget_df_ytday[budget_df_ytday['세부사업명'].str.contains(freq_kwd)]['세부사업코드']) # 빈도수 키워드 필터링

In [26]:
# 종합쇼핑몰 진행건 제외 & 지출액0(미진행건) 추가
budget_df_today_temp1 = budget_df_today[~budget_df_today['세부사업코드'].isin(set(bid_cd_split_today + bid_cd_freq_today))]
budget_df_today_temp2 = budget_df_today[budget_df_today['세부사업코드'].isin(set(bid_cd_split_today + bid_cd_freq_today))]
budget_df_today_temp2 = budget_df_today_temp2[budget_df_today_temp2['지출액']==0].reset_index(drop=True)
budget_df_today_fin = pd.concat([budget_df_today_temp1, budget_df_today_temp2], axis=0).reset_index(drop=True)

# budget_df_ytday_temp1 = budget_df_ytday[~budget_df_ytday['세부사업코드'].isin(set(bid_cd_split_ytday + bid_cd_freq_ytday))]
# budget_df_ytday_temp2 = budget_df_ytday[budget_df_ytday['세부사업코드'].isin(set(bid_cd_split_ytday + bid_cd_freq_ytday))]
# budget_df_ytday_temp2 = budget_df_ytday_temp2[budget_df_ytday_temp2['지출액']==0].reset_index(drop=True)
# budget_df_ytday_fin = pd.concat([budget_df_ytday_temp1, budget_df_ytday_temp2], axis=0).reset_index(drop=True)

In [27]:
# 중요 사업 체크
budget_df_today_final = pd.concat([budget_df_today_fin[budget_df_today_fin['세부사업명'].str.contains('인조잔디')],
                                   budget_df_today_fin[~budget_df_today_fin['세부사업명'].str.contains('인조잔디')]],axis=0).reset_index(drop=True)

# budget_df_ytday_final = pd.concat([budget_df_ytday_fin[budget_df_ytday_fin['세부사업명'].str.contains('인조잔디')],
#                                    budget_df_ytday_fin[~budget_df_ytday_fin['세부사업명'].str.contains('인조잔디')]],axis=0).reset_index(drop=True)

In [28]:
# 최종데이터셋
budget_df_today_final = budget_df_today_final[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)
budget_df_delete = budget_df_delete[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)
budget_df_new = budget_df_new[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)

In [30]:
# 빅쿼리에 적재
save_dataframe_to_bigquery(budget_df_today_final,'budget','budget_df_listup',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_delete,'budget','budget_df_delete',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_new,'budget','budget_df_new',bigquerykey_path)

Data inserted into table budget_df_listup successfully.
Data inserted into table budget_df_delete successfully.
Data inserted into table budget_df_new successfully.


#### 교육청 예산서

In [31]:
# 빅쿼리에서 불러오기
edu_budget_df = get_dataframe_from_bigquery('edu','edu_budget',bigquerykey_path)
edu_budget_df.sort_values('시도').reset_index(drop=True)

,시도,시군구,구분,과업명,금액,면적,예산집행
0,강원특별자치도,화천교육지원청,공립,상서중학교 운동장 보수,"89,533,000","4,664",학교
1,강원특별자치도,인제교육지원청,공립,상남중학교 개방형 운동장 조성,"725,302,000","6,507",교육청
2,강원특별자치도,강원특별자치도교육청,공립,강릉제일고등학교 축구장 인조잔디 교체,"1,020,560,000","8,576",교육청
3,강원특별자치도,강릉교육지원청,공립,관동중학교 인조잔디 운동장 조성,"806,400,000","4,082",교육청
4,강원특별자치도,강원특별자치도교육청,사립,동해광희고등학교 인조잔디운동장 보수,"51,700,000",None,None
...,...,...,...,...,...,...,...
189,충청남도,보령교육지원청,공립,성주초등학교 운동장 교체,"70,980,000",None,None
190,충청남도,당진교육지원청,공립,면천중학교 운동장 교체,"297,561,000",None,None
191,충청북도,충청북도교육청,공립,충북산업과학고등학교 다목적구장 조성,"769,799,000","5,540",교육청
192,충청북도,청주교육지원청,공립,서경중학교 운동장 보수 및 바닥교체,"602,871,000","3,500",교육청


In [32]:
edu_budget_df['과업명_re'] = edu_budget_df['과업명'].apply(clean_text)

In [33]:
edu_budget_df

,시도,시군구,구분,과업명,금액,면적,예산집행,과업명_re
0,전북특별자치도,전북특별자치도교육청,None,완주 스포츠클럽 실외야구장,"1,689,820,000",None,None,완주 스포츠클럽 실외야구장
1,경기도,경기도교육청,None,친환경 운동장 조성사업비 지원 (30교),"15,000,000,000",None,학교,친환경 운동장 조성사업비 지원 교
2,전라남도,전남교육청,None,운동장 개보수(고2),"1,652,038,000",None,학교,운동장 개보수고
3,울산광역시,울산광역시교육청,None,친환경운동장조성 다목적구장 중학교 1교,"14,000,000",None,학교,친환경운동장조성 다목적구장 중학교 교
4,울산광역시,울산광역시교육청,None,친환경운동장조성 다목적구장 고등학교 6교,"465,600,000",None,학교,친환경운동장조성 다목적구장 고등학교 교
...,...,...,...,...,...,...,...,...
189,경상남도,의령교육지원청,사립,정곡중학교 체육관 신축(계속비),"38,000,000",None,학교,정곡중학교 체육관 신축계속비
190,대구광역시,대구광역시교육청,사립,대륜고등학교 축구장 인조잔디 재조성,"578,358,000","7,447",학교,대륜고등학교 축구장 인조잔디 재조성
191,울산광역시,울산광역시교육청,사립,친환경운동장조성 다목적구장 중학교 (사립) 1교,"169,141,500",None,학교,친환경운동장조성 다목적구장 중학교 사립 교
192,전북특별자치도,전북특별자치도교육청,사립,익산고등학교 인조잔디 조성,"1,050,001,000","7,417",교육청,익산고등학교 인조잔디 조성
